#Máquinas de Boltzmann Restringidas

# Instalamos pytorch

In [1]:
#pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Clonamos el repositorio para obtener el dataset

In [2]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 10153 (delta 25), reused 39 (delta 16), pack-reused 10096
Receiving objects: 100% (10153/10153), 236.95 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Checking out files: 100% (10108/10108), done.


In [3]:
# Importar las librerías
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importar el dataset

In [4]:
movies = pd.read_csv("/content/deeplearning-az/datasets/Part 5 - Boltzmann Machines (BM)/ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("/content/deeplearning-az/datasets/Part 5 - Boltzmann Machines (BM)/ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("/content/deeplearning-az/datasets/Part 5 - Boltzmann Machines (BM)/ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')


# Preparar el conjunto de entrenamiento y elconjunto de testing

In [5]:
training_set = pd.read_csv("/content/deeplearning-az/datasets/Part 5 - Boltzmann Machines (BM)/ml-100k/u1.base", sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("/content/deeplearning-az/datasets/Part 5 - Boltzmann Machines (BM)/ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

# Obtener el número de usuarios y de películas

In [6]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna

In [7]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
training_set = convert(training_set)
test_set = convert(test_set)


# Convertir los datos a tensores de Torch

In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)

In [10]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [11]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Crear la arquitectura de la Red Neuronal (Modelo Probabilistico Gráfico)

In [12]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):           #x = mini_batch_size x nv
        wx = torch.mm(x, self.W.t()) #mini_batch_size x nh
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):           #y = mini_batch_size x nh
        wy = torch.mm(y, self.W) #mini_batch_size x nv
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)   
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [13]:
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)

# Entrenar la RBM


In [14]:
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    training_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print("Epoch: "+str(epoch)+", Loss: "+str(training_loss/s))

Epoch: 1, Loss: tensor(0.3574)
Epoch: 2, Loss: tensor(0.2614)
Epoch: 3, Loss: tensor(0.2306)
Epoch: 4, Loss: tensor(0.2481)
Epoch: 5, Loss: tensor(0.2637)
Epoch: 6, Loss: tensor(0.2278)
Epoch: 7, Loss: tensor(0.2520)
Epoch: 8, Loss: tensor(0.2434)
Epoch: 9, Loss: tensor(0.2467)
Epoch: 10, Loss: tensor(0.2458)


# Testear la RBM

In [15]:
testing_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        testing_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
        print("Testing Loss: "+str(testing_loss/s))


Testing Loss: tensor(0.2701)
Testing Loss: tensor(0.2714)
Testing Loss: tensor(0.2963)
Testing Loss: tensor(0.2472)
Testing Loss: tensor(0.2621)
Testing Loss: tensor(0.2431)
Testing Loss: tensor(0.2370)
Testing Loss: tensor(0.2117)
Testing Loss: tensor(0.1993)
Testing Loss: tensor(0.1927)
Testing Loss: tensor(0.2051)
Testing Loss: tensor(0.1976)
Testing Loss: tensor(0.2070)
Testing Loss: tensor(0.2060)
Testing Loss: tensor(0.2225)
Testing Loss: tensor(0.2165)
Testing Loss: tensor(0.2234)
Testing Loss: tensor(0.2167)
Testing Loss: tensor(0.2105)
Testing Loss: tensor(0.2227)
Testing Loss: tensor(0.2297)
Testing Loss: tensor(0.2349)
Testing Loss: tensor(0.2316)
Testing Loss: tensor(0.2297)
Testing Loss: tensor(0.2237)
Testing Loss: tensor(0.2254)
Testing Loss: tensor(0.2319)
Testing Loss: tensor(0.2299)
Testing Loss: tensor(0.2341)
Testing Loss: tensor(0.2337)
Testing Loss: tensor(0.2386)
Testing Loss: tensor(0.2361)
Testing Loss: tensor(0.2410)
Testing Loss: tensor(0.2369)
Testing Loss: 